In [ ]:
%pip install -q keras_nlp
%pip install -q tflite_runtime
%pip install -q einops

import gc
interpolated_length = 19 #mean length /2

In [ ]:
import math
import time
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from os.path import join as pjoin

ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

def tf_get_features(ftensor):
    def feat_wrapper(ftensor):
        return load_relevant_data_subset(ftensor.numpy().decode('utf-8'))
    return tf.py_function(
        feat_wrapper,
        [ftensor],
        Tout=tf.float32
    )

def tf_nan_mean(x, axis=0, keepdims=True):
    return (tf.reduce_sum(
        tf.where(tf.math.is_nan(x), tf.zeros_like(x), x), 
        axis=axis, keepdims=keepdims) 
        / tf.reduce_sum(
            tf.where(
                tf.math.is_nan(x), tf.zeros_like(x), tf.ones_like(x)), 
            axis=axis, keepdims=keepdims))

def tf_nan_std(x, axis=0, keepdims=True):
    d = x - tf_nan_mean(x, axis=axis, keepdims=keepdims)
    return tf.math.sqrt(tf_nan_mean(d * d, axis=axis, keepdims=keepdims))

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)

def timeSpent(since):
    now = time.time()
    s = now - since
    return asMinutes(s)

class TimeLimitCallback(tf.keras.callbacks.Callback):
    def __init__(self, start_time, max_duration_hours=8, max_duration_minutes=30):
        super(TimeLimitCallback, self).__init__()
        self.start_time = start_time
        self.max_duration_seconds = max_duration_hours * 3600 + max_duration_minutes * 60

    def on_train_batch_end(self, batch, logs=None):
        elapsed_time = time.time() - self.start_time
        if elapsed_time > self.max_duration_seconds:
            self.model.stop_training = True
            print(f"Training stopped: time limit of {self.max_duration_seconds/3600:.1f} hours exceeded")

@tf.autograph.experimental.do_not_convert
def detuple(v, l, g, pid, sid):
    return (v, l)

@tf.autograph.experimental.do_not_convert
def ensure(shape):
    return lambda x : tf.ensure_shape(x, shape)

def scheduler(epoch, lr):
    if epoch < 8:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
def create_load_tfrecords(new=False):
    idx = 0
    while f"run_{idx}" in os.listdir('.'):
        idx +=1

    if new:
        path = f"run_{idx}"
        print(f"Results will be saved at path {path}")
        os.makedirs(path)
        # !mkdir $path

    else:
        path = f"run_{idx-1}"
        print(f"Using path {path}")
    
    return pjoin(path, 'full_cv.tfrecord')

In [ ]:
import tensorflow as tf
tfkl = tf.keras.layers

#import imageio
#from PIL import Image
#import matplotlib.pyplot as plt

class Preprocess(tf.keras.layers.Layer):
    '''
    This is the first version of Preprocessing layer with visualisation of preprocessed data as gifs.
    - 4 modalities : pose, eyes, mouth, and normalized hand.
    - Each modality is individually time-interpolated according to interpolated_length, at the timesteps 
        having data for this modality.
    - A problem might happen for samples having less than 'interpolated_length' available timesteps.
    - Besides, it might be hard for the model to account for different modalities sampled at different
        timesteps.
    '''
    def __init__(self, interpolated_length=10):
        super(Preprocess, self).__init__()
        self.fixed_frames = interpolated_length

        self.mod2indexes = {
            "pose": tf.constant([468, 500, 501, 502, 503, 522]), 
            "hands": tf.constant([
                [468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488], 
                [522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542]
            ]), 
            "eyes": tf.constant([7, 33, 133, 144, 145, 153, 154, 155, 157, 158, 159, 160, 161, 163, 173, 246, 249, 263, 362, 373, 374, 380, 381, 382, 384, 385, 386, 387, 388, 390, 398, 466]), 
            "mouth": tf.constant([13, 14, 78, 80, 81, 82, 87, 88, 95, 178, 191, 308, 310, 311, 312, 317, 318, 324, 402, 415])
        }

        self.lh_idx_range=(468, 489)
        self.rh_idx_range=(522, 543)

        self.mod2edge = {
            "pose": np.array([
                [4, 3, 1, 0, 2], #for right handed
                [4, 2, 1, 3, 0]  #for left handed
            ]),
            "hands": np.array([
                [0, 1, 2, 3, 4],
                [0, 5, 6, 7, 8],
                [0, 17, 18, 19, 20],
                [17, 13, 14, 15, 16],
                [5, 9, 10, 11, 12],
            ]), 
            "eyes": np.array([
                [1, 15, 12, 11, 10, 9, 8, 14, 2, 7, 6, 5, 4, 3, 13, 0, 1],
                [18, 30, 24, 25, 26, 27, 28, 31, 17, 16, 29, 19, 20, 21, 22, 23, 18]
            ]),
            "mouth": np.array([
                [2, 10, 3, 4, 5, 0, 14, 13, 12, 19, 11, 17, 16, 18, 15, 1, 6, 9, 7, 8]
            ])
        }

        self.mod2style = {
            "pose": ('black', 3),
            "hands": ('blue', 2), 
            "eyes": ('black', 1),
            "mouth": ('red', 1)
        }
    
    @tf.function
    def pad_or_truncate_center(self, tensor, fixed_length):
        seq_length = tf.shape(tensor)[0]
        
        if seq_length > fixed_length:
            # Truncate the sequence by selecting the center patch
            start_idx = (seq_length - fixed_length) // 2
            tensor = tensor[start_idx:start_idx + fixed_length, :, :]
        elif seq_length < fixed_length:
            # Pad the sequence with zeros
            padding_length = fixed_length - seq_length
            padding = tf.zeros((padding_length, tf.shape(tensor)[1], tf.shape(tensor)[2]))
            tensor = tf.concat([tensor, padding], axis=0)

        return tensor

    def call(self, tensor):
        tensor = tensor[:, :, :2]  # drop z axis
        do_symetry = (tf.reduce_mean(tf.cast(tf.math.is_nan(tensor[:, self.rh_idx_range[0]:self.rh_idx_range[1], :]), tf.float32))
                    < tf.reduce_mean(tf.cast(tf.math.is_nan(tensor[:, self.lh_idx_range[0]:self.lh_idx_range[1], :]), tf.float32)))
        
        tensor = tf.cond(# symetry on 2nd axis when left-handed
            tf.equal(do_symetry, True),
            lambda: tf.stack([1 - tensor[:, :, 0], tensor[:, :, 1]], axis=-1),
            lambda: tensor
        )
        #if do_symetry:  # symetry on 2nd axis when left-handed
        #    tensor = tf.stack([1 - tensor[:, :, 0], tensor[:, :, 1]], axis=-1)

        modalities, mod_indexes = {}, {}
        for modality, modalitidx in self.mod2indexes.items():
            mod_tensor = tf.gather(tensor, modalitidx, axis=1)
            fallback_shape = tf.shape(mod_tensor)
            
            if modality == "pose":
                mod_tensor = tf.cond(
                    do_symetry,
                    lambda: mod_tensor[:, 1:],
                    lambda: mod_tensor[:, :-1]
                )
            if modality == "hands":
                mod_tensor = tf.cond(
                    tf.equal(do_symetry, True),
                    lambda: mod_tensor[:, 1],
                    lambda: mod_tensor[:, 0]
                )

            #if modality == "pose":
            #    mod_tensor = mod_tensor[:, 1:] if do_symetry else mod_tensor[:, :-1]
            #elif modality == "hands":
            #    mod_tensor = mod_tensor[:, tf.cast(do_symetry, tf.int32)]

            # Replacing loop with tensor operations
            nan_frames_mask = tf.reduce_all(tf.math.is_nan(mod_tensor), axis=[1, 2])
            nan_frames_mask = tf.reshape(nan_frames_mask, [-1])
            
            mod_tensor = tf.boolean_mask(mod_tensor, tf.math.logical_not(nan_frames_mask), axis=0)
            mod_tensor = tf.where(tf.math.is_nan(mod_tensor), tf.zeros_like(mod_tensor), mod_tensor)

            # if tf.shape(mod_tensor)[0] == 0:
            #     fallback_shape = tf.concat([tf.constant([1]), fallback_shape[1:]], axis=0)
            #     mod_tensor = tf.zeros(fallback_shape)

            # mod_n = tf.shape(mod_tensor)[0]
            # mod_n = tf.reduce_max(tf.stack([mod_n, 1]))
            # mod_n = tf.stack([self.fixed_frames - 1, mod_n])
            # mod_n = tf.reduce_min(mod_n)
            # mod_time_interpolated = tf.cast(tf.linspace(0, mod_n - 1, self.fixed_frames), tf.int32)

            # mod_tensor = tf.gather(mod_tensor, mod_time_interpolated, axis=0)

            mod_tensor = self.pad_or_truncate_center(mod_tensor, self.fixed_frames)

            modalities[modality] = mod_tensor
            # mod_indexes[modality] = mod_time_interpolated

        return modalities
        # return modalities, mod_indexes
  
    def plot_sketch(self, array, modality):
        modalitidx = self.mod2edge[modality]
        col, width = self.mod2style[modality]
        for j, seg in enumerate(modalitidx):
            seg = np.array(seg)
            plt.plot(array[seg, 0], -array[seg, 1], '-', color=col, linewidth=width)    
        
    def output_gif_result(self, tensor, out_path='untitled.gif', gif_size=(224, 224)):
        is_right_handed = (tf.reduce_mean(tf.cast(tf.math.is_nan(tensor[0, self.rh_idx_range[0]:self.rh_idx_range[1], :2]), tf.float32)) 
                        < tf.reduce_mean(tf.cast(tf.math.is_nan(tensor[0, self.lh_idx_range[0]:self.lh_idx_range[1], :2]), tf.float32)))
        
        mod2tensor = self.call(tensor)[0]
        mod2tensor = {k:v.numpy() for k,v in mod2tensor.items()}

        for pose, hands in zip(mod2tensor['pose'], mod2tensor['hands']):
            if is_right_handed:
                pose[-1] = hands[0]
                self.mod2edge["pose"] = self.mod2edge["pose"][:1]
            else:
                pose[0] = hands[0]
                self.mod2edge["pose"] = self.mod2edge["pose"][-1:]

        images = []

        times = [{} for _ in range(self.fixed_frames)]
        for mod, arrays in mod2tensor.items():
            for t, array in enumerate(arrays):
                times[t][mod] = array

        for i, time in enumerate(times):
            plt.figure(figsize=(gif_size[0]/100, gif_size[1]/100))
            for modality in time.keys():
                modalitidx = self.mod2edge[modality]
                col, width = self.mod2style[modality]
                array = time[modality]
                for j, seg in enumerate(modalitidx):
                    seg = np.array(seg)
                    plt.plot(array[seg, 0], -array[seg, 1], '-', color=col, linewidth=width)
            plt.xlim(-0.25, 1.25)
            plt.ylim(-1.5, 0)
            plt.axis('off')
            plt.gca().set_position([0, 0, 1, 1])
            plt.savefig(f"tmp/frame_{i:02d}.png")
            plt.close()
            images.append(Image.open(f"tmp/frame_{i:02d}.png"))

        images[0].save(
            out_path,
            save_all=True,
            append_images=images[1:],
            duration=100,
            loop=0,
            size=gif_size)
        
        # Save the images as a GIF file
        imageio.mimsave(out_path, images, fps=5)
        return out_path

In [ ]:
from os.path import join as pjoin
from os.path import exists
import os
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import json

try :
    BASE_DIR = "/kaggle/input/asl-signs/"
    df = pd.read_csv(pjoin(BASE_DIR, "train.csv"))
except :
    BASE_DIR = "asl-signs"
    df = pd.read_csv(pjoin(BASE_DIR, "train.csv"))

path2label = dict(zip(df.path, df.sign))
label2int = json.load(open(pjoin(BASE_DIR, "sign_to_prediction_index_map.json"), 'rb'))

# KFold
def get_KFold_dataset(preprocessing, path='untitled', n_splits=3):
    dir_path = path
    path = pjoin(path, 'full_cv.tfrecord')

    def parse_function(example_proto):
        feature_description = {
            'pose': tf.io.FixedLenFeature(shape=(interpolated_length, 5, 2), dtype=tf.float32),
            'hands': tf.io.FixedLenFeature(shape=(interpolated_length, 21, 2), dtype=tf.float32),
            'eyes': tf.io.FixedLenFeature(shape=(interpolated_length, 32, 2), dtype=tf.float32),
            'mouth': tf.io.FixedLenFeature(shape=(interpolated_length, 20, 2), dtype=tf.float32),

            # 'pose_idx': tf.io.FixedLenFeature(shape=(10), dtype=tf.int64),
            # 'hands_idx': tf.io.FixedLenFeature(shape=(10), dtype=tf.int64),
            # 'eyes_idx': tf.io.FixedLenFeature(shape=(10), dtype=tf.int64),
            # 'mouth_idx': tf.io.FixedLenFeature(shape=(10), dtype=tf.int64),
            
            'label': tf.io.FixedLenFeature(shape=(), dtype=tf.int64),
            'group': tf.io.FixedLenFeature(shape=(), dtype=tf.int64),
            'pid': tf.io.FixedLenFeature(shape=(), dtype=tf.int64),
            'sid': tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
        }
        parsed_example = tf.io.parse_single_example(example_proto, feature_description)
        return parsed_example, parsed_example['label']

    if exists(dir_path):
        print(f"Reloading dataset from path {path}")
    else:
        print(f"Dataset will be saved at path {path}")
        os.makedirs(dir_path)

        ds = tf.data.TFRecordDataset(path)
        X_ds = tf.data.Dataset.from_tensor_slices(
            BASE_DIR + "/" + df.path.values
            ).map(tf_get_features)
        y_ds = tf.data.Dataset.from_tensor_slices(
            df.sign.map(label2int).values.reshape(-1,1)
            )
        
        # Perform stratisfied kfold split
        sgkf = StratifiedGroupKFold(n_splits=n_splits, random_state=42, shuffle=True)

        fold2id = dict()
        for fold_idx, (index_train, index_valid) in enumerate(sgkf.split(df.path, df.sign, df.participant_id)):
            fold2id[fold_idx] = np.unique(df.participant_id.values[index_valid])
            
        id2fold = dict()
        for k,v in fold2id.items():
            for vv in v:
                id2fold[vv] = k

        g_ds = tf.data.Dataset.from_tensor_slices(
            df.participant_id.map(id2fold).values.reshape(-1, 1)
        )
        pid_ds = tf.data.Dataset.from_tensor_slices(
            df.participant_id.values
        )
        sid_ds = tf.data.Dataset.from_tensor_slices(
            df.sequence_id.values
        )
        
        with tf.io.TFRecordWriter(path) as writer:
            zipper = zip(X_ds.map(preprocessing), y_ds, g_ds, pid_ds, sid_ds)
            for example in zipper:
                X, y, g, pid, sid = example
                # X, posX = X
                feature = {
                    'pose': tf.train.Feature(float_list=tf.train.FloatList(value=X["pose"].numpy().flatten())),
                    'hands': tf.train.Feature(float_list=tf.train.FloatList(value=X["hands"].numpy().flatten())),
                    'eyes': tf.train.Feature(float_list=tf.train.FloatList(value=X["eyes"].numpy().flatten())),
                    'mouth': tf.train.Feature(float_list=tf.train.FloatList(value=X["mouth"].numpy().flatten())),

                    # 'pose_idx': tf.train.Feature(int64_list=tf.train.Int64List(value=posX["pose"].numpy().flatten())),
                    # 'hands_idx': tf.train.Feature(int64_list=tf.train.Int64List(value=posX["hands"].numpy().flatten())),
                    # 'eyes_idx': tf.train.Feature(int64_list=tf.train.Int64List(value=posX["eyes"].numpy().flatten())),
                    # 'mouth_idx': tf.train.Feature(int64_list=tf.train.Int64List(value=posX["mouth"].numpy().flatten())),

                    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=y.numpy().flatten())),
                    'group': tf.train.Feature(int64_list=tf.train.Int64List(value=g.numpy().flatten())),
                    'pid': tf.train.Feature(int64_list=tf.train.Int64List(value=pid.numpy().flatten())),
                    'sid': tf.train.Feature(int64_list=tf.train.Int64List(value=sid.numpy().flatten())),
                }
                example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
                writer.write(example_proto.SerializeToString())

    ds = tf.data.TFRecordDataset(path)
    ds = ds.map(parse_function).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
from keras_nlp.layers.transformer_encoder import TransformerEncoder
from keras_nlp.layers import TokenAndPositionEmbedding
from einops import rearrange
import tensorflow as tf
tfkl = tf.keras.layers

def get_embedder_model(d_model, dropout): 
    #may be better to optimize on the hidden dimension for each modality
    return tf.keras.Sequential([
        tfkl.TimeDistributed(tfkl.Flatten()),
        tfkl.Dense(2*d_model),
        tfkl.BatchNormalization(),
        tfkl.Dropout(dropout),
        tfkl.Activation('relu'),
        tfkl.Dense(2*d_model),
        tfkl.BatchNormalization(),
        tfkl.Dropout(dropout),
        tfkl.Activation('relu'),
        tfkl.Dense(d_model),
        tfkl.BatchNormalization(),
        tfkl.Dropout(dropout),
        tfkl.Activation('relu')
    ])

from keras import backend as backend
def get_model(hp):
    backend.clear_session()
    # 0 mask, 1 eyes, 2 hands, 3 mouth, 4 pose, 5 cls
    modalities = ['eyes', 'hands', 'mouth', 'pose']

    mod2inputs = {
        'eyes': tfkl.Input(shape=(interpolated_length, 32, 2), name="eyes"), 
        'hands': tfkl.Input(shape=(interpolated_length, 21, 2), name="hands"), 
        'mouth': tfkl.Input(shape=(interpolated_length, 20, 2), name="mouth"), 
        'pose': tfkl.Input(shape=(interpolated_length, 5, 2), name="pose")
    }

    # attention mask
    attention_mask = tf.concat([tf.math.reduce_any(tf.math.logical_not(tf.math.equal(mod2inputs[mod], 0)), axis=[-2, -1]) for mod in modalities], axis=-1)
    attention_mask = tf.concat([tf.tile(tf.constant([True]), (tf.shape(attention_mask)[0],))[:, tf.newaxis], attention_mask], axis=-1)

    mod2model = {k:get_embedder_model(hp['dim_model'], hp['emb_dropout']) for k in modalities}
    modalities_embedding = tf.stack([mod2model[mod](mod2inputs[mod]) for mod in modalities], axis=1)

    # mod-space & time encoding
    tokens = [k*tf.ones_like(mod2inputs[mod][:, :, 0, 0]) for k,mod in enumerate(modalities)]
    tokens = tf.stack(tokens, axis=1)

    tokens_embedder = TokenAndPositionEmbedding(6, interpolated_length, hp['dim_model'], mask_zero=True)
    tokens_embedding = tokens_embedder(tokens)

    # add to embedding
    modalities_ts = modalities_embedding + tokens_embedding
    modalities_ts = rearrange(modalities_ts, 'b m t d -> b (m t) d')

    cls = tokens_embedder(tf.constant([5]))
    cls = tf.tile(cls, (tf.shape(modalities_ts)[0], 1))[:, tf.newaxis]
    full_embedding = tf.concat([cls, modalities_ts], axis=1)

    for _ in range(hp['num_enc']):
        full_embedding = TransformerEncoder(intermediate_dim=hp['dim_model'], num_heads=hp['nhead'], dropout=hp['t_dropout'])(full_embedding, 
                                                                                                                              padding_mask=attention_mask)

    pool = full_embedding[:, 0]

    clas = tfkl.Dense(250, 'softmax')(pool)

    model = tf.keras.Model(inputs=list(mod2inputs.values()), outputs=clas)
    return model

In [ ]:
from os.path import join as pjoin
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from tqdm import tqdm

import tensorflow as tf
import tensorflow_addons as tfa
tfkl = tf.keras.layers
from tqdm.keras import TqdmCallback
import json
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
import time
import math
import os

import warnings
warnings.simplefilter('ignore')

In [ ]:
mock_run = False

In [ ]:
path = "preprocessed_datasets/vivit_experiment_centerpad/"

if not mock_run:
    ds = get_KFold_dataset(Preprocess(interpolated_length), path)
    ds = ds.shuffle(int(1e5)).batch(256)
print('et zebardi')

hp = {
    'emb_dropout':0.1,
    'dim_model':196,#177
    'num_enc':3,
    # 'ff_dim':1024,
    'nhead':8,
    't_dropout':0.3
}

fold_idx = 0
#train_ds = ds.filter(lambda X, y: X['group'] != fold_idx).shuffle(int(1e5)).batch(256)
#valid_ds = ds.filter(lambda X, y: X['group'] == fold_idx).batch(256)

In [ ]:
class TimeLimitCallback(tf.keras.callbacks.Callback):
    def __init__(self, start_time, max_duration_hours=8, max_duration_minutes=30):
        super(TimeLimitCallback, self).__init__()
        self.start_time = start_time
        self.max_duration_seconds = max_duration_hours * 3600 + max_duration_minutes * 60

    def on_train_batch_end(self, batch, logs=None):
        elapsed_time = time.time() - self.start_time
        if elapsed_time > self.max_duration_seconds:
            self.model.stop_training = True
            print(f"Training stopped: time limit of {self.max_duration_seconds/3600:.1f} hours exceeded")

In [ ]:
import time

folds = 3

for fold_idx in range(folds):
    model = get_model(hp)
    start = time.time()

    lr = 1e-3

    model.compile(
        tf.keras.optimizers.Adam(learning_rate=lr),
        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            tf.keras.metrics.SparseCategoricalAccuracy(),
        ]
    )
    
    if not mock_run:
        hist = model.fit(
        x=ds,
        epochs=60,
        verbose=2, # 2 = one line per epoch
        callbacks=[
            TimeLimitCallback(start, 2, 30), #1 minute max for testing purposes
            tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.ModelCheckpoint(pjoin(path.split('/')[0], f"model_{fold_idx}"), 
                save_best_only=True, 
                save_weights_only=True,
                restore_best_weights=True, 
                monitor="sparse_categorical_accuracy", mode="max"),
            tf.keras.callbacks.EarlyStopping(patience=10, 
             monitor="sparse_categorical_accuracy", mode="max", restore_best_weights=True)
            ],
        #validation_data=valid_ds,
        #validation_freq=1,
        workers=2,
        use_multiprocessing=True
    )

In [ ]:
gc.collect()

440 s cpu

In [ ]:
def get_inference_model():
    inputs = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")
    
    diffs = Preprocess(interpolated_length)(inputs)
    diffs = {k:tf.expand_dims(v, 0) for k,v in diffs.items()}
    
    models = [get_model(hp) for _ in range(folds)]
    for fold_idx in range(folds):
        if not mock_run:
            models[fold_idx].load_weights(pjoin(path.split('/')[0], f"model_{fold_idx}"))
        models[fold_idx]._name += f"_{fold_idx}" 
    outputs = [model(diffs) for model in models]
    vector = tf.reduce_mean(outputs, axis=0) #average the models

    output = tfkl.Activation(activation="linear", name="outputs")(vector)
    inference_model = tf.keras.Model(inputs=inputs, outputs=output) 
    inference_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
    return inference_model

inference_model = get_inference_model()
#inference_model.summary()
params = inference_model.count_params()
print(params/1e6)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
tflite_model = converter.convert()
model_path = "model.tflite"
# Save the model.
with open(model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
!zip submission.zip $model_path

In [ ]:
BASE_DIR = "/kaggle/input/asl-signs/"
train_df = pd.read_csv(pjoin(BASE_DIR, "train.csv"))

path2label = dict(zip(train_df.path, train_df.sign))
label2int = json.load(open(pjoin(BASE_DIR, "sign_to_prediction_index_map.json"), 'rb'))

int2label = {v:k for k,v in label2int.items()}

import tflite_runtime.interpreter as tflite
interpreter = tflite.Interpreter(model_path)
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")
#for i in range(len(train_df)):
for i in range(50):
    frames = load_relevant_data_subset(f'/kaggle/input/asl-signs/{train_df.iloc[i].path}')
    output = prediction_fn(inputs=frames)
    sign = np.argmax(output["outputs"])
    print(f"Predicted label: {int2label[sign]}, Actual Label: {train_df.iloc[i].sign} (shape {frames.shape})")